## Coarse-Grained Stoichiometry of OXPHOS and Growth reactions
To align the stoichiometries of the reduced HT29 network with the Shestov et al. model, we follow Baroukh et al. (2014) and Tummler et al. (2015) to collapse the elementary flux modes for respiration and biomass formation into two single coarse grained reactions, using pFBA solution as a reference. For growth, we require the reduced model to match the full‐model maximal growth rate on the defined medium at a lactate secretion–to–glucose uptake flux ratio of 1.5.

In [ ]:
import cobra

pruned_model = cobra.io.read_sbml_model('../../model/ht29_pruned.xml')
pruned_model.solver = 'gurobi'


In [ ]:
from cobra.flux_analysis import pfba

# check the default pFBA solution
with pruned_model as m:
    solution = pfba(m)

    summary = m.summary(solution)
    print(summary.to_string(names=True))


In [ ]:
import math
from cobra import Reaction


def build_balancing_reaction(S, flux_vector, rxn_id, model, tol = 1e-10):
    """
    Constructs a single lumped reaction that captures the net effect of all fluxes not explicitly modeled in the reduced network.
    
    Parameters:
    - S: numpy array of shape (m, n), the stoichiometric matrix of the model (m metabolites, n reactions).
    - flux_vector: pandas Series of length n containing optimized fluxes from pFBA.
    - rxn_id: string identifier for the new reaction.
    - model: cobra.Model
    - tol: float threshold; coefficients with absolute value below tol are omitted.
    
    Returns:
    - reaction: cobra.Reaction representing a coarse-grained reaction for the dynamic model, preserving mass and cofactor balances.
    """
 
    result = S.dot(flux_vector)
    metabolites = {}

    for i in range(0, len(result)):
        metabolite = model.metabolites[i]

        if not math.isclose(result[i], 0, abs_tol=tol):
            metabolites[metabolite] = result[i].round(2)

    reaction = Reaction(rxn_id)
    reaction.add_metabolites(metabolites)
    return reaction


In [ ]:
from cobra.flux_analysis import pfba
from cobra.util import create_stoichiometric_matrix

# Reactions explicitly retained in the reduced network (from Shestov et al. with additional transport reactions)
protected_reactions = {'MAR09034', 'MAR05029', 'MAR09048', 'MAR04896',
                       'MAR04394', 'MAR04381', 'MAR04379', 'MAR04375',
                       'MAR04391', 'MAR04373', 'MAR04368', 'MAR04365',
                       'MAR04363', 'MAR04358', 'MAR04388', 'MAR05998',
                       'MAR09135', 'MAR03964', 'MAR10024'}

# Compute OXPHOS reaction stoichiometry
with pruned_model as m:
    # Set objective to ATP maintenance reaction
    m.objective = 'MAR03964'
    # Define minimal medium: 
    # MAR09034: Glucose uptake, MAR09048: Oxygen uptake
    m.medium = {'MAR09034': 1, 'MAR09048': 1000}

    solution = pfba(m)
    v = solution.fluxes.copy()
    # Zero out protected reactions
    v[v.index.isin(protected_reactions)] = 0
    # Scale fluxes for consistency with the Shestov model
    v = v / 2

    S = create_stoichiometric_matrix(pruned_model, array_type='dense')
    oxphos = build_balancing_reaction(S, v, 'OXPHOS', m)
    
    print("Oxidative Phosphorylation Reaction:")
    print(oxphos.build_reaction_string(use_metabolite_names=True))

pruned_model.add_reactions([oxphos])
protected_reactions.add('OXPHOS')


In [ ]:
with pruned_model as m:
    solution = pfba(m)

    summary = m.summary(solution)
    print(summary.to_string(names=True))
    

In [ ]:
import numpy as np

# Sweep oxygen uptake bounds to visualize growth and L/G ratio
o2_bounds = np.linspace(-1.0, -0.1, num=10)

growth_rates = []
l_g_ratios = []

for bound in o2_bounds:
    with pruned_model as m:
        m.reactions.MAR09048.bounds = (bound, 0)
        sol = pfba(m)
        mu = sol.fluxes['MAR10024']
        ratio = -sol.fluxes['MAR09135'] / sol.fluxes['MAR09034']
        growth_rates.append(mu)
        l_g_ratios.append(ratio)
    

In [ ]:
import matplotlib.pyplot as plt

# Plot growth rate and L/G ratio vs oxygen bound on dual axes
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

ax1.plot(o2_bounds, growth_rates, marker='o', label='Growth rate')
ax2.plot(o2_bounds, l_g_ratios, marker='s', label='L/G ratio', linestyle='--')

ax1.set_xlabel('Oxygen uptake boundary (mmol/gDW/h)')
ax1.set_ylabel('Growth rate (1/h)')
ax2.set_ylabel('Lactate-to-Glucose flux ratio')

lines_1, labels_1 = ax1.get_legend_handles_labels()
lines_2, labels_2 = ax2.get_legend_handles_labels()
ax1.legend(lines_1 + lines_2, labels_1 + labels_2, loc='lower right')

plt.grid(False)
plt.show()


In [ ]:
from cobra.flux_analysis import pfba

# Compute Growth reaction stoichiometry
with pruned_model as m:
    # Constrain oxygen uptake to get L/G flux ratio of 1.5
    m.reactions.MAR09048.bounds = -0.53, 0

    solution = pfba(m)
    ratio = -solution.fluxes['MAR09135'] / solution.fluxes['MAR09034']
    print(f"Lactate-to-Glucose flux ratio: {ratio:.1f}\n")
    
    v = solution.fluxes.copy()
    v[v.index.isin(protected_reactions)] = 0
    v = v * 100 / solution.fluxes['MAR10024']

    S = create_stoichiometric_matrix(m, array_type='dense')
    growth_reaction = build_balancing_reaction(S, v, 'Growth', m)
    
    print("Growth Reaction Stoichiometry:")
    print(growth_reaction.build_reaction_string(use_metabolite_names=True))
